In [221]:
from pycocotools.coco import COCO
import json
import os

folder = 'val'

dataDir='../../datasets/car_damage/'
anotherDatasetDir = os.path.join(dataDir, f'another_dataset/{folder}/')
newFilename = os.path.join(dataDir, folder, f'COCO_new_{folder}_annos.json')

annoPath = os.path.join(dataDir, folder, f'COCO_{folder}_annos.json')
annoFile = open(annoPath)
coco = COCO(annoPath)
data = json.load(annoFile)
print('Images before', len(data['images']))
print('Annotations before', len(data['annotations']))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Images before 11
Annotations before 24


In [222]:
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
import numpy as np

coco_img = coco.loadImgs(0)[0]
coco_anno = coco.loadAnns(0)[0]
print(coco_img)
print(coco_anno)
polygon = np.array(coco_anno['segmentation'][0]).reshape((len(coco_anno['segmentation'][0])//2,2))
polygon = tuple(map(tuple, polygon))

image = Image.open(os.path.join(dataDir, folder, coco_img['file_name']))
draw = ImageDraw.Draw(image)
draw.polygon(polygon, fill=200)
# image.show()

{'coco_url': '', 'date_captured': '2020-07-14 09:59:34.190485', 'file_name': '1.jpg', 'flickr_url': '', 'height': 1024, 'id': 0, 'license': 1, 'width': 1024}
{'area': 26268.00000000003, 'bbox': [364, 143, 190, 490], 'category_id': 1, 'id': 0, 'image_id': 0, 'iscrowd': 0, 'segmentation': [[454, 143, 388, 296, 364, 424, 375, 505, 422, 597, 495, 633, 516, 577, 554, 489, 539, 349, 550, 259, 539, 210, 521, 166, 495, 150]]}


In [223]:
from datetime import datetime

now = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

In [224]:
f = open(os.path.join(anotherDatasetDir, 'via_region_data.json'))
anotherData = json.load(f)

anotherData.pop('test_pattern_qbist.jpg129855', None)
anotherData.pop('a_swan_swimming_in_geneve_lake.jpg62201', None)
anotherData.pop('sinus_test_pattern.jpg27894', None)

lastImgId = coco.getImgIds()[-1]
lastAnnoId = coco.getAnnIds()[-1]
print(lastImgId, lastAnnoId)

for key, value in anotherData.items():
    lastImgId += 1

    pilImage = Image.open(os.path.join(
        anotherDatasetDir, value['filename']))
    draw = ImageDraw.Draw(pilImage)
    w, h = pilImage.size

    image = {'coco_url': '', 'data_captured': now, 'file_name': value['filename'], 'flickr_url': '',
            'height': h, 'id': lastImgId, 'license': 1, 'width': w}

    for annoId in value['regions']:
        lastAnnoId += 1
        anno = value['regions'][annoId]['shape_attributes']
        # print(anno)
        segmentation = []

        bbox = ((min(anno['all_points_x']), min(anno['all_points_y'])), (max(anno['all_points_x']), max(anno['all_points_y'])))
        draw.rectangle(bbox)
        bbox = [bbox[0][0], bbox[0][1], bbox[1][0]-bbox[0][0], bbox[1][1]-bbox[0][1]]
        for pointId in range(len(anno['all_points_x'])):
            x, y = anno['all_points_x'][pointId], anno['all_points_y'][pointId]
            segmentation.append(x)
            segmentation.append(y)

        anno = {'area': -1, 'bbox': bbox, 'category_id': 1, 'id': lastAnnoId, 'image_id': lastImgId, 'iscrowd': 0, 'segmentation': [segmentation]}
        data['annotations'].append(anno)
        # draw.polygon(segmentation, fill=200)
        # print(anno)

    # print(image)
    # pilImage.show()
    data['images'].append(image)


10 23


In [225]:
with open(newFilename, 'w') as outfile:
    json.dump(data, outfile, sort_keys=True, indent=4)
newCoco = COCO(newFilename)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [226]:
print('Images after ', len(newCoco.getImgIds()))
print('Annotations after', len(newCoco.getAnnIds()))

Images after  26
Annotations after 43
